In [1]:
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import FAISS

import os
import pickle

In [2]:
# Load index from file

In [3]:
load_dotenv()

True

In [4]:
loaded_faiss_vs = FAISS.load_local(
    folder_path="./vectordb/faiss/pubmedqa/",
    embeddings=OpenAIEmbeddings())

In [5]:
retriever = loaded_faiss_vs.as_retriever(search_kwargs={'k': 5})

In [6]:
# Define the RAG pipeline

In [7]:
model = 'gpt-3.5-turbo-instruct'
llm = OpenAI(model_name=model, openai_api_key=os.getenv("OPENAI_API_KEY"))

In [8]:
template = """Answer the question or Explain the topic given this additional context: {context}
Question: {question}"""

In [9]:
prompt = ChatPromptTemplate.from_template(template)

In [10]:
docs_file_path = './dataset/pubmedqa/documents.pkl'
with open(docs_file_path, 'rb') as file:
    docs = pickle.load(file)
print("Documents loaded successfully.")

Documents loaded successfully.


In [11]:
def format_docs(_docs):
    ls = []
    for doc in _docs:
        if doc.page_content in docs:
            ls.append(docs[doc.page_content]["text"][:800])
    return ls

In [12]:
chain = ({"context": retriever | format_docs, "question": RunnablePassthrough()} 
         | prompt 
         | llm 
         | StrOutputParser())

In [13]:
# Run the RAG pipeline

In [14]:
query_file_path = './openai_embeddings/pubmedqa/query_embeddings.pkl'
with open(query_file_path, 'rb') as file:
    loaded_queries = pickle.load(file)
print("Query embeddings loaded successfully.")

Query embeddings loaded successfully.


In [15]:
loaded_queries["21645374"]["text"]

'Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?'

In [16]:
chain.invoke(loaded_queries["21645374"]["text"])

'\n\nAnswer: Yes, the role of mitochondria during programmed cell death (PCD) has been recognized in animals, and it has been less studied during PCD in plants. The paper mentioned in the additional context elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. This study suggests that mitochondria may be involved in the remodelling of lace plant leaves during PCD.'

In [17]:
# loaded_queries["PLAIN-23"]["text"]

In [18]:
# chain.invoke(loaded_queries["PLAIN-23"]["text"])

In [19]:
# loaded_queries["PLAIN-33"]["text"]

In [20]:
# chain.invoke(loaded_queries["PLAIN-33"]["text"])

In [21]:
# loaded_queries["PLAIN-78"]["text"]

In [22]:
# chain.invoke(loaded_queries["PLAIN-78"]["text"])

In [23]:
# loaded_queries["PLAIN-143"]["text"]

In [24]:
# chain.invoke(loaded_queries["PLAIN-143"]["text"])

In [25]:
# loaded_queries["PLAIN-165"]["text"]

In [26]:
# chain.invoke(loaded_queries["PLAIN-165"]["text"])

In [27]:
# llm("If I have inflammatory bowel disease, how will smoking affect my risk of developing colorectal cancer?")

In [28]:
# chain.invoke("If I have inflammatory bowel disease, how will smoking affect my risk of developing colorectal cancer?")
